DIABETIES

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
dataset= pd.read_csv(r'C:\Users\Pranshu Saini\Desktop\disease-prediction-main\docpat\datasets\diabetes.csv')
dataset.head(5)
dataset.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [4]:
# removing highly correlated features

corr_matrix = dataset.corr().abs() 

mask = np.triu(np.ones_like(corr_matrix, dtype = bool))
tri_df = corr_matrix.mask(mask)

to_drop = [x for x in tri_df.columns if any(tri_df[x] > 0.92)]

df = dataset.drop(to_drop, axis = 1)

print(f"The reduced dataframe has {df.shape[1]} columns.")

The reduced dataframe has 9 columns.


In [5]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [ ]:
A= dataset.drop(columns = 'Outcome', axis=1)
B= dataset['Outcome']
A_training, A_testing, B_training, B_testing = train_test_split(A,B, test_size = 0.2, stratify=B, random_state=5)
print(A.shape, A_training.shape, A_testing.shape)


(768, 8) (614, 8) (154, 8)


LogisticRegression

In [ ]:
# fitting data to model

from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(A_training, B_training)
B_pred = log_reg.predict(A_testing)
# accuracy score

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(accuracy_score(B_training, log_reg.predict(A_training)))

log_reg_acc = accuracy_score(B_testing, log_reg.predict(A_testing))
print(log_reg_acc)

0.7817589576547231
0.7532467532467533


c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


K Neighbors Classifier (KNN)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(A_training, B_training)
# model predictions 

B_pred = knn.predict(A_testing)
# accuracy score

print(accuracy_score(B_training, knn.predict(A_training)))

knn_acc = accuracy_score(B_testing, knn.predict(A_testing))
print(knn_acc)

0.7980456026058632
0.7142857142857143


Support Vector Machine (SVM)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

svc = SVC(probability=True)
parameters = {
    'gamma' : [0.0001, 0.001, 0.01, 0.1],
    'C' : [0.01, 0.05, 0.5, 0.1, 1, 10, 15, 20]
}

grid_search = GridSearchCV(svc, parameters)
grid_search.fit(A_training, B_training)
# best parameters

grid_search.best_params_



{'C': 1, 'gamma': 0.0001}

In [ ]:
# best score 

grid_search.best_score_

0.7557643609222977

In [ ]:
svc = SVC(C = 10, gamma = 0.01, probability=True)
svc.fit(A_training, B_training)
# model predictions 

B_pred = svc.predict(A_testing)
# accuracy score

print(accuracy_score(B_training, svc.predict(A_training)))

svc_acc = accuracy_score(B_testing, svc.predict(A_testing))
print(svc_acc)
# classification report

print(classification_report(B_testing, B_pred))

1.0
0.6428571428571429
              precision    recall  f1-score   support

           0       0.66      0.93      0.77       100
           1       0.46      0.11      0.18        54

    accuracy                           0.64       154
   macro avg       0.56      0.52      0.48       154
weighted avg       0.59      0.64      0.56       154



DECISION TREE

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()

parameters = {
    'criterion' : ['gini', 'entropy'],
    'max_depth' : range(2, 32, 1),
    'min_samples_leaf' : range(1, 10, 1),
    'min_samples_split' : range(2, 10, 1),
    'splitter' : ['best', 'random']
}

grid_search_dt = GridSearchCV(dtc, parameters, cv = 5, n_jobs = -1, verbose = 1)
grid_search_dt.fit(A_training, B_training)

Fitting 5 folds for each of 8640 candidates, totalling 43200 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(2, 32),
                         'min_samples_leaf': range(1, 10),
                         'min_samples_split': range(2, 10),
                         'splitter': ['best', 'random']},
             verbose=1)

In [ ]:
# best score

grid_search_dt.best_score_
dtc = DecisionTreeClassifier(criterion= 'entropy', max_depth= 19, min_samples_leaf= 4, min_samples_split= 6, splitter= 'random')
dtc.fit(A_training, B_training)

DecisionTreeClassifier(criterion='entropy', max_depth=19, min_samples_leaf=4,
                       min_samples_split=6, splitter='random')

In [ ]:
B_pred = dtc.predict(A_testing)
# accuracy score

print(accuracy_score(B_training, dtc.predict(A_training)))

dtc_acc = accuracy_score(B_testing, dtc.predict(A_testing))
print(dtc_acc)

0.8224755700325733
0.6883116883116883


In [ ]:
# classification report

print(classification_report(B_testing, B_pred))

              precision    recall  f1-score   support

           0       0.74      0.80      0.77       100
           1       0.57      0.48      0.52        54

    accuracy                           0.69       154
   macro avg       0.65      0.64      0.64       154
weighted avg       0.68      0.69      0.68       154



In [ ]:
models = pd.DataFrame({
    'Model': ['Logistic Regression', 'KNN', 'SVM', 'Decision Tree Classifier'],
    'Score': [100*round(log_reg_acc,4), 100*round(knn_acc,4), 100*round(svc_acc,4), 100*round(dtc_acc,4)]
})
models.sort_values(by = 'Score', ascending = False)

,Model,Score
0,Logistic Regression,75.32
1,KNN,71.43
3,Decision Tree Classifier,68.83
2,SVM,64.29


In [ ]:
import pickle
filename = 'C:/Users/Dell/OneDrive/Desktop/DM PROJECT/diabetes_model.pkl'
pickle.dump(log_reg, open(filename, 'wb'))

In [ ]:
'''import pickle
def load_model(path):
    with open(path, 'rb') as file:
        model = pickle.load(file)
diabetes_model = load_model(r'C:\Users\DELL\Desktop\app\diabetes_model.pkl')
def predict(inputs):
    return diabetes_model.predict(inputs)'''